In [3]:
# !pip install git+https://hf.co/ctheodoris/Geneformer.git
# !pip install anndata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 4.5 MB/s eta 0:00:00


In [2]:
from geneformer.pretrainer import token_dictionary

In [3]:
from geneformer.tokenizer import GENE_MEDIAN_FILE
import pickle


with open(GENE_MEDIAN_FILE, "rb") as f:
        gene_median_dict = pickle.load(f)

In [27]:
import pandas as pd

sequence_path = '/home/ec2-user/cytoself-data/sequences.csv'
sequences = pd.read_csv(sequence_path, index_col=0)
sequences['emebed_index'] = range(len(sequences))

In [5]:
sequences

,Peptide,localization,loc
ensg,,,
ENSG00000075624,MDDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVG...,other,8
ENSG00000100591,MAKWGEGDPRWIVEERADATNVNNWHWTERDASNWSTDKLKTLFLA...,other,8
ENSG00000156735,MSALRRSGYGPSDGPSYGRYYGPGGGDVPVHPPPPLYPLRPEPPQP...,other,8
ENSG00000229524,MEPNDSTSTAVEEPDSLEVLVKTLDSQTRTFIVGAQMNVKEFKEHI...,other,8
ENSG00000105401,MVDYSVWDHIEVSDDEDETHPNIDTASLFRWRHQARVERMEQFQKE...,other,8
...,...,...,...
ENSG00000174749,MCSAGELLRGGDGGERDEDGDALAEREAAGTGWDPGASPRRRGQRP...,ER,0
ENSG00000138439,MSQRVRRNGSPTPAGSLGGGAVATAGGPGSRLQPMRATVPFQLKQQ...,other,8
ENSG00000116001,MEDEMPKTLYVGNLSRDVTEALILQLFSQIGPCKNCKMIMDTAGND...,other,8


In [6]:
tokens = [token_dictionary[seq] if seq in token_dictionary else 1 for seq in sequences.index]
med_values = [gene_median_dict[seq] if seq in token_dictionary else 1.0 for seq in sequences.index]
missing = [seq for seq in sequences.index if seq not in token_dictionary.keys()]

In [7]:
from transformers import AutoModelForMaskedLM

In [8]:
model = AutoModelForMaskedLM.from_pretrained("ctheodoris/Geneformer")
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(25426, 256, padding_idx=0)
      (position_embeddings): Embedding(2048, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.02, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.02, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine

In [9]:
import torch

tokens = torch.asarray(tokens).unsqueeze(0)
embeddings_X = model.bert.embeddings.word_embeddings(tokens)
embeddings_X = embeddings_X.squeeze(0).detach().numpy()

In [10]:
embeddings_X[1]

array([-0.1347448 ,  0.02516621, -0.08930477, -0.22753006, -0.0536321 ,
       -0.15104474, -0.62907225, -0.5620529 , -0.5545428 , -0.15731956,
       -0.06326873, -0.16703615,  0.07761274, -0.4826781 ,  0.27704725,
       -0.4814318 , -0.02782201, -0.12473135, -0.2918584 , -0.23595989,
        0.13122381, -0.362167  , -0.15300746, -0.51428837,  0.70991737,
        0.22127354,  0.37203762, -0.03250565,  0.22285426, -0.2915918 ,
       -0.16916119, -0.40502438, -0.11491598,  0.39278442, -0.12747583,
        0.11583631, -0.29056096,  0.25671843,  0.04997071,  0.01382954,
       -0.3243846 , -0.16010804,  0.12388009,  0.01506513, -0.4213195 ,
       -0.16838686,  0.14468949, -0.19207011, -0.34768307, -0.31091997,
       -0.42770454, -0.13368294, -0.26299444, -0.03754232, -0.2636267 ,
       -0.05608178, -0.24258058,  0.08936351,  0.29139987,  0.03150004,
        0.16226164,  0.3751811 ,  0.2827999 ,  0.15538642, -0.4578103 ,
       -0.3393625 , -0.4956451 , -1.0524172 , -0.20800568,  0.04

In [15]:
# import numpy as np

# geneformer_embeddings_path = '/home/ec2-user/cytoself-data/geneformer_embeds.npy'
# np.save(geneformer_embeddings_path, embeddings)

## Using abundance too

In [28]:
abundance_path = '/home/ec2-user/cytoself-data/opencell-protein-abundance.csv'

abundance = pd.read_csv(abundance_path, index_col='ensg_id')

In [29]:
abundance

,gene_name,enst_id,uniprot_id,protein_name,hek_rna_tpm,hek_protein_conc_nm,hek_protein_copy_number
ensg_id,,,,,,,
ENSG00000121410,A1BG,ENST00000263100,P04217,Alpha-1B-glycoprotein,28.903340,NaN,NaN
ENSG00000148584,A1CF,ENST00000374001,Q9NQ94,APOBEC1 complementation factor,0.012306,NaN,NaN
ENSG00000175899,A2M,ENST00000318602,P01023,Alpha-2-macroglobulin,0.070627,0.088404,53.219474
ENSG00000166535,A2ML1,ENST00000299698,A8K2U0,Alpha-2-macroglobulin-like protein 1,0.181935,NaN,NaN
ENSG00000184389,A3GALT2,ENST00000442999,U3KPV4,"Alpha-1,3-galactosyltransferase 2",0.068718,NaN,NaN
...,...,...,...,...,...,...,...
ENSG00000070476,ZXDC,ENST00000389709,Q2QGD7,Zinc finger protein ZXDC,24.693054,9.350763,5629.159502
ENSG00000203995,ZYG11A,ENST00000371528,Q6WRX3,Protein zyg-11 homolog A,9.269409,3.946253,2375.644235
ENSG00000162378,ZYG11B,ENST00000294353,Q9C0D3,Protein zyg-11 homolog B,14.993380,79.729615,47997.228471


In [30]:
results = sequences.join(abundance, how='outer')

In [31]:
results

,Peptide,localization,loc,emebed_index,gene_name,enst_id,uniprot_id,protein_name,hek_rna_tpm,hek_protein_conc_nm,hek_protein_copy_number
ENSG00000000003,NaN,NaN,NaN,NaN,TSPAN6,ENST00000373020,O43657,Tetraspanin-6,78.173345,220.396131,132678.470862
ENSG00000000005,NaN,NaN,NaN,NaN,TNMD,ENST00000373031,Q9H2S6,Tenomodulin,0.024793,NaN,NaN
ENSG00000000419,NaN,NaN,NaN,NaN,DPM1,ENST00000371588,O60762,Dolichol-phosphate mannosyltransferase subunit 1,70.163048,291.860863,175700.239406
ENSG00000000457,NaN,NaN,NaN,NaN,SCYL3,ENST00000367772,Q8IZE3,Protein-associating with the carboxyl-terminal...,6.691786,102.417278,61655.201416
ENSG00000000460,NaN,NaN,NaN,NaN,C1orf112,ENST00000413811,Q9NSG2,Uncharacterized protein C1orf112,34.080587,29.256358,17612.327504
...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288701,NaN,NaN,NaN,NaN,PRRC2B,ENST00000357304,Q5JSZ5,Protein PRRC2B,117.041816,95.653974,57583.692444
ENSG00000288702,NaN,NaN,NaN,NaN,UGT1A3,ENST00000482026,P35503,UDP-glucuronosyltransferase 1A3,0.000000,NaN,NaN
ENSG00000288705,NaN,NaN,NaN,NaN,UGT1A5,ENST00000373414,P35504,UDP-glucuronosyltransferase 1A5,0.000000,NaN,NaN
ENSG00000288722,NaN,NaN,NaN,NaN,F8A1,ENST00000610495,P23610,40-kDa huntingtin-associated protein,14.971270,108.449987,65286.892174


In [32]:
import numpy as np

results.loc[:, 'geneformer_token'] = [token_dictionary[seq] if seq in token_dictionary else 1 for seq in results.index]
results.loc[:, 'geneformer_median_value'] = [gene_median_dict[seq] if seq in token_dictionary else np.NaN for seq in results.index]

In [33]:
import numpy as np

results.loc[:, 'geneformer_normalized_abundence'] = np.divide(results['hek_rna_tpm'], results['geneformer_median_value'] )

In [35]:
ordered = results['geneformer_normalized_abundence'].sort_values(ascending=False).index
# ordered = sequences.index

In [36]:
ordered

Index(['ENSG00000228253', 'ENSG00000181163', 'ENSG00000243678',
       'ENSG00000198563', 'ENSG00000184110', 'ENSG00000074800',
       'ENSG00000196262', 'ENSG00000133124', 'ENSG00000132341',
       'ENSG00000132507',
       ...
       'ENSG00000288513', 'ENSG00000288547', 'ENSG00000288611',
       'ENSG00000288642', 'ENSG00000288649', 'ENSG00000288701',
       'ENSG00000288702', 'ENSG00000288705', 'ENSG00000288722',
                     nan],
      dtype='object', length=19288)

In [143]:
big = 2048 - len(sequences)
keep_genes = list(range(big)) + list(big + np.where(~results.loc[ordered, 'emebed_index'][big:].isna())[0])

ordered_tokens = results.loc[ordered, 'geneformer_token'].values[keep_genes]
ordered_tokens = torch.asarray(ordered_tokens).unsqueeze(0)
# output = model.bert(input_ids=ordered_tokens).last_hidden_state
# output = model.bert.embeddings(ordered_tokens)
# output = model.bert.embeddings.word_embeddings(ordered_tokens)

output = model.bert.embeddings(ordered_tokens)
for layer in model.bert.encoder.layer[:-1]:
    output = layer(output)[0]
output = output.squeeze(0).detach().numpy()

# embed_indices = results.loc[ordered, 'emebed_index'].values
# inv_embed_indices = np.zeros_like(embed_indices)
# inv_embed_indices[embed_indices] = np.arange(len(embed_indices))
# embeddings = output[inv_embed_indices]

embeddings = np.zeros((len(sequences), output.shape[1]))
for out, index in zip(output, results.loc[ordered, 'emebed_index'].values[keep_genes]):
    if ~np.isnan(index):
        embeddings[int(index)] = out

In [144]:
import numpy as np

# geneformer_embeddings_path = '/home/ec2-user/cytoself-data/geneformer_word_embeddings.npy'
geneformer_embeddings_path = '/home/ec2-user/cytoself-data/geneformer_best_embeddings.npy'
# geneformer_embeddings_path = '/home/ec2-user/cytoself-data/geneformer_last_hidden_state_embeddings.npy'
np.save(geneformer_embeddings_path, embeddings)

In [ ]:
## 